In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/02 17:24:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.5.5'

In [5]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-02 17:25:44--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.46, 18.155.128.6, 18.155.128.222, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M   162MB/s    in 0.4s    

2025-03-02 17:25:45 (162 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [6]:
!ls -lh yellow_tripdata_2024-10.parquet

-rw-rw-r-- 1 viktorija viktorija 62M Dec 18 21:21 yellow_tripdata_2024-10.parquet


In [7]:
schema = types.StructType([
    types.StructField('VendorID', types.IntegerType(), True),
    types.StructField('tpep_pickup_datetime', types.TimestampType(), True),
    types.StructField('tpep_dropoff_datetime', types.TimestampType(), True),
    types.StructField('passenger_count', types.LongType(), True),
    types.StructField('trip_distance', types.DoubleType(), True),
    types.StructField('RatecodeID', types.LongType(), True),
    types.StructField('store_and_fwd_flag', types.StringType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('payment_type', types.LongType(), True),
    types.StructField('fare_amount', types.DoubleType(), True),
    types.StructField('extra', types.DoubleType(), True),
    types.StructField('mta_tax', types.DoubleType(), True),
    types.StructField('tip_amount', types.DoubleType(), True),
    types.StructField('tolls_amount', types.DoubleType(), True),
    types.StructField('improvement_surcharge', types.DoubleType(), True),
    types.StructField('total_amount', types.DoubleType(), True),
    types.StructField('congestion_surcharge', types.DoubleType(), True),
    types.StructField('Airport_fee', types.DoubleType(), True)
])

In [8]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .parquet('yellow_tripdata_2024-10.parquet')

df = df.repartition(4)

df.write.parquet('data/pq/yellow/2024/10/', mode='overwrite')

In [10]:
!ls -lh data/pq/yellow/2024/10/

total 100M
-rw-r--r-- 1 viktorija viktorija   0 Mar  2 17:26 _SUCCESS
-rw-r--r-- 1 viktorija viktorija 25M Mar  2 17:26 part-00000-4d943173-e007-450c-ad87-5fae92f5efb0-c000.snappy.parquet
-rw-r--r-- 1 viktorija viktorija 25M Mar  2 17:26 part-00001-4d943173-e007-450c-ad87-5fae92f5efb0-c000.snappy.parquet
-rw-r--r-- 1 viktorija viktorija 25M Mar  2 17:26 part-00002-4d943173-e007-450c-ad87-5fae92f5efb0-c000.snappy.parquet
-rw-r--r-- 1 viktorija viktorija 25M Mar  2 17:26 part-00003-4d943173-e007-450c-ad87-5fae92f5efb0-c000.snappy.parquet


In [11]:
df = spark.read.parquet('data/pq/yellow/2024/10/')

In [12]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



**Q3**: How many taxi trips were there on October 15?

In [13]:
from pyspark.sql import functions as F

In [14]:
df \
    .withColumn('pickup_date', F.to_date(df.tpep_pickup_datetime)) \
    .filter("pickup_date = '2024-10-15'") \
    .count()

128893

In [15]:
df.registerTempTable('yellow_2024_10')

/home/viktorija/spark/spark-3.5.5-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [16]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    yellow_2024_10
WHERE
    to_date(tpep_pickup_datetime) = '2024-10-15';
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



**Q4**: Longest trip

In [17]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [18]:
spark.sql("""
SELECT
    CAST(tpep_dropoff_datetime AS LONG) - CAST(tpep_pickup_datetime AS LONG) as duration,
    ROUND((CAST(tpep_dropoff_datetime AS LONG) - CAST(tpep_pickup_datetime AS LONG)) / 3600, 1) AS hour    
FROM 
    yellow_2024_10
ORDER BY
    1 DESC
LIMIT 1;
""").show()

+--------+-----+
|duration| hour|
+--------+-----+
|  585424|162.6|
+--------+-----+



**Q6**: Least frequent pickup location zone

In [19]:
df_zones = spark.read.parquet('zones')

In [20]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [22]:
df_zones.registerTempTable('zones')

In [23]:
spark.sql("""
SELECT
    yel.PULocationID, pul.Zone, 
    COUNT(1) as count_trips
FROM 
    yellow_2024_10 yel LEFT JOIN zones pul ON yel.PULocationID = pul.LocationID
GROUP BY 
    1, 2
ORDER BY
    3 ASC
LIMIT 5;
""").show()

+------------+--------------------+-----------+
|PULocationID|                Zone|count_trips|
+------------+--------------------+-----------+
|         105|Governor's Island...|          1|
|           5|       Arden Heights|          2|
|         199|       Rikers Island|          2|
|         111| Green-Wood Cemetery|          3|
|           2|         Jamaica Bay|          3|
+------------+--------------------+-----------+

